In [8]:
import os
import pandas as pd
import glob
import menpo.io as mio
from menpodetect import load_opencv_frontal_face_detector, load_opencv_profile_face_detector
from menpowidgets import visualize_images

In [9]:
front = load_opencv_frontal_face_detector()
side = load_opencv_profile_face_detector()

In [67]:
annos = pd.DataFrame.from_csv('gold.tsv',header=0,sep="\t")
#annos

In [87]:
height = 227
width = 227
channels = 3

images = []
labels = []
for fp in glob.glob('./img/*.JPG'):
    fn = os.path.basename(fp)
    labels.append(annos.loc[annos['file'] == fn]['ow'].sum())
    image = mio.import_image(fp)
    front(image)
    if(image.n_landmark_groups == 0):
        side(image)
    if(image.n_landmark_groups > 0):
        image = image.crop_to_landmarks()
    else:
        cropwidth = min(image.height, image.width) / 2
        x1 = image.height - cropwidth
        image = image.crop((image.height / 2 - cropwidth, image.width / 2 - cropwidth), 
                           (image.height / 2 + cropwidth, image.width / 2 + cropwidth))
    image = image.resize((height, width))
    images.append(image)
    mio.export_image(image, "./cropped/" + fn, "jpg", overwrite=True)

#visualize_images(images)


In [89]:
import numpy as np
np.random.seed(1337)  # for reproducibility

# returns (X_train, y_train), (X_test, y_test)
def randomSplit(X, y, propTest):
    assert X.shape[0] == y.shape[0]
    p = np.random.permutation(len(y))
    sX, sy = X[p], y[p]
    cutoff = round(propTest * len(y))
    return (sX[cutoff:], sy[cutoff:]), (sX[0:cutoff], sy[0:cutoff])


In [95]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution3D, MaxPooling3D
from keras.utils import np_utils
from keras import backend as K

batch_size = 64
nb_classes = 3
nb_epoch = 12

# input image dimensions
img_rows, img_cols, img_channels = height, width, channels
# number of convolutional filters to use
nb_filters = 96
# size of pooling area for max pooling
pool_size = (3, 3)
# convolution kernel size
kernel_size = (7, 7)

Xs = np.stack([img.pixels for img in images])
if K.image_dim_ordering() == 'tf':
    Xs = Xs.reshape(Xs.shape[0], img_rows, img_cols, channels)
    input_shape = input_shape = (img_rows, img_cols, channels)
else:
    input_shape = (channels, img_rows, img_cols)
    
ys = np.array(labels)
(X_train, y_train), (X_test, y_test) = randomSplit(Xs, ys, 0.3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to trinary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

print('y_train shape:', y_train.shape)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])



X_train shape: (6, 227, 227, 3)
6 train samples
3 test samples
y_train shape: (6, 3)
Train on 6 samples, validate on 3 samples
Epoch 1/12
6/6 [==============================] - 53s - loss: 1.0998 - acc: 0.1667 - val_loss: 10.4255 - val_acc: 0.3333
Epoch 2/12
6/6 [==============================] - 48s - loss: 2.6864 - acc: 0.8333 - val_loss: 10.4361 - val_acc: 0.3333
Epoch 3/12
6/6 [==============================] - 60s - loss: 1.6928 - acc: 0.8333 - val_loss: 0.9659 - val_acc: 0.6667
Epoch 4/12
6/6 [==============================] - 45s - loss: 1.4251 - acc: 0.0000e+00 - val_loss: 3.5597 - val_acc: 0.3333
Epoch 5/12
6/6 [==============================] - 47s - loss: 0.6232 - acc: 0.8333 - val_loss: 1.0839 - val_acc: 0.0000e+00
Epoch 6/12
6/6 [==============================] - 46s - loss: 1.0256 - acc: 0.6667 - val_loss: 2.2008 - val_acc: 0.3333
Epoch 7/12
6/6 [==============================] - 45s - loss: 0.5842 - acc: 0.8333 - val_loss: 0.7967 - val_acc: 0.6667
Epoch 8/12
6/6 [=======